In [ ]:
from exiftool import ExifToolHelper
from os import path, listdir, remove, walk
from concurrent.futures import ThreadPoolExecutor
from hashlib import md5
from collections import defaultdict
from datetime import datetime

In [ ]:
# !sudo apt install exiftool

In [ ]:
test_pic = '/mnt/nas/pics_test_date_correct/2023-05-31/IMG_4744.MOV'

In [ ]:
test_pic = '/mnt/nas/Pictures/2009-12-21/IMG_0244.MOV'

In [ ]:
test_pic = '/mnt/nas/imports_gopro/iphone_18MAY/190APPLE/IMG_0739.AAE'

In [ ]:
test_pic = '/mnt/nas/Pictures/00_no_date_found/IMG_0278.AAE'

In [ ]:
with ExifToolHelper() as et:
    output = et.get_metadata(test_pic)[0]
output

In [ ]:
output['File:FileType']

In [ ]:
date_tags = [
            'QuickTime:CreateDate',
            'QuickTime:CreationDate',
            'EXIF:CreateDate',
            'EXIF:DateTimeOriginal',
            'File:ModificationDate',  # Skip because throws no date for .AAE files
            'File:FileModifyDate',
            # 'File:FileAccessDate',
            # 'File:FileInodeChangeDate'
        ]

In [ ]:
[date_tags[5]]

In [ ]:
images_list = [path.join(dir,x) for x in listdir(dir) if not x.startswith('.')]
images_list

In [ ]:
len(images_list)

In [ ]:
def get_date(item):
    date = False
    try:
        with ExifToolHelper() as et:
            output = et.get_metadata(item)[0]
    except:
        return(item)
    if output.get('EXIF:CreateDate') is not None:
        date = output.get('EXIF:CreateDate')
    elif output.get('QuickTime:CreateDate') is not None:
        date = output.get('QuickTime:CreateDate')
    elif output.get('File:FileModifyDate') is not None:
        date = output.get('File:FileModifyDate')
    
    if date == False:
        return None
    else:
        return date

In [ ]:
def get_media_date(file):
        date = None
        try:
            with ExifToolHelper() as et:
                output = et.get_metadata(file)[0]
        except:
            return None
        
        # Handle video files specifically
        # if file.lower().endswith(('.mov', '.mp4', '.avi')):  # Add other video formats as needed
        if 'QuickTime:CreationDate' in output:
            print('creationdate')
            date_str = output['QuickTime:CreationDate']
            # Parse the date and time along with the timezone
            try:
                date = datetime.strptime(date_str, '%Y:%m:%d %H:%M:%S%z').astimezone()
            except ValueError:
                date = None
        elif 'QuickTime:CreateDate' in output:
            print('createdate')
            date_str = output['QuickTime:CreateDate']
            # Parse the date and time along with the timezone
            try:
                date = datetime.strptime(date_str, '%Y:%m:%d %H:%M:%S')
            except ValueError:
                date = None
        elif'EXIF:CreateDate' in output:
                print('second')
                date_str = output['EXIF:CreateDate']
                try:
                    date = datetime.strptime(date_str, '%Y:%m:%d %H:%M:%S')
                except ValueError:
                    date = None
        elif'EXIF:DateTimeOriginal' in output:
                print('third')
                date_str = output['EXIF:DateTimeOriginal']
                try:
                    date = datetime.strptime(date_str, '%Y:%m:%d %H:%M:%S')
                except ValueError:
                    date = None
        elif output.get('File:FileModifyDate'):
            print('foruth')
            date = output.get('File:FileModifyDate')
            date = datetime.strptime(date, '%Y:%m:%d %H:%M:%S%z').astimezone()
        else:
            date = None
            
        # Return the date or none
        if date:
            return date.strftime('%Y-%m-%d %H:%M:%S')
        else:
            return None

In [ ]:
get_media_date(test_pic)

In [ ]:
test_date = get_date(test_pic)

In [ ]:
datetime.strptime(test_date, '%Y:%m:%d %H:%M:%S').astimezone()

In [ ]:
with ThreadPoolExecutor() as executor:
    results = list(executor.map(get_date, images_list))

In [ ]:
len(results)

In [ ]:
results

In [ ]:
from datetime import datetime

In [ ]:
new_date = datetime.strptime(results[0].split(' ')[0], "%Y:%m:%d")

In [ ]:
new_date.strftime("%Y-%m-%d")
                    

In [ ]:
[x for x in results if 'pictures' in x]

In [ ]:
for img in images_list:
    date = False
    with ExifToolHelper() as et:
        output = et.get_metadata(img)[0]
    if output.get('EXIF:CreateDate') is not None:
        date = True
    elif output.get('QuickTime:CreateDate') is not None:
        date = True
    elif output.get('File:FileModifyDate') is not None:
        date = True
    
    if date == False:
        print(img)
    

# Remove duplicates from a folder

In [ ]:
def find_and_remove_duplicate_files(directory):
    # Create a dictionary to store files with the same initial 8 characters
    file_dict = {}
    
    for root, _, files in walk(directory):
        for filename in files:
            # Extract the first 8 characters of the filename
            key = filename[:8]
            full_path = path.join(root, filename)
            
            if key in file_dict:
                # A file with the same first 8 characters exists, check for duplicate content
                existing_file = file_dict[key]
                
                # Calculate the hash of the existing file
                existing_hash = md5(open(existing_file, 'rb').read()).hexdigest()
                
                # Calculate the hash of the current file
                current_hash = md5(open(full_path, 'rb').read()).hexdigest()
                
                if existing_hash == current_hash:
                    # If hashes are the same, remove the current file
                    print(f"Removing duplicate: {full_path}")
                    # remove(full_path)
                else:
                    # If hashes are different, add this file to the dictionary
                    file_dict[key] = full_path
            else:
                # No file with the same initial 8 characters, add to the dictionary
                file_dict[key] = full_path


In [ ]:
# Usage example
directory_path = '/mnt/nas/TEST_SORT_OUTPUT/2021-07-24/'
find_and_remove_duplicate_files(directory_path)

In [ ]:
def find_and_remove_duplicates(directory):
    """
    Find and remove duplicate files from a list based on their content hash and extension.

    Args:
        file_list (list): A list of file names to be processed.

    Returns:
        list: The list with duplicate files removed.
    """
    
    images_list = [path.join(directory,x) for x in listdir(directory) if not x.startswith('.')]
    images_list.sort()
    
    # Create a dictionary to store files by their (hash, extension) tuple
    file_dict = {}
    cleaned_list = []

    for filename in images_list:
        # Extract the file extension
        file_ext = filename.split('.')[-1]

        # Calculate the hash of the current file
        file_hash = md5(open(filename, 'rb').read()).hexdigest()

        # Create a (hash, extension) tuple to check for duplicates
        file_tuple = (file_hash, file_ext)

        if file_tuple not in file_dict:
            # If this (hash, extension) combination is not in the dictionary, add it
            file_dict[file_tuple] = filename
            cleaned_list.append(filename)
        else:
            # If it already exists, it's a duplicate, so don't add it to the cleaned list
            print(f"Removing duplicate: {filename}")

    return cleaned_list

In [ ]:
# Remove duplicate files and store the cleaned list
cleaned_list = find_and_remove_duplicates(directory_path)

# Print the cleaned list
# print("Cleaned list:")
# for filename in cleaned_list:
#     print(filename)

In [ ]:
listdir('/mnt/nas/TEST_SORT_OUTPUT/2021-07-24/')

In [ ]:
dir_check_path = '/mnt/nas/TEST_SORT_OUTPUT/2021-07-24/'
list_dir_check = [path.join(dir_check_path,x) for x in listdir(dir_check_path) if not x.startswith('.')]
list_dir_check

In [ ]:
for index, item in enumerate(list_dir_check):
    key = item.split(" ")[0]
    same_filename = [x for x in list_dir_check if key in x]
    same_filename.sort()
    if len(same_filename) > 1:
        print(same_filename)
        temp_list = []
        for file in same_filename:
            
        if index == 2:
            break
        
            
    

In [ ]:
# Function to calculate the hash of a file
def hash_file(file_path):
    hasher = md5()
    with open(file_path, 'rb') as f:
        while True:
            data = f.read(65536)  # Read in 64k chunks
            if not data:
                break
            hasher.update(data)
    return hasher.hexdigest()

# Directory containing your files
directory = '/mnt/nas/TEST_SORT_OUTPUT/2021-07-24/'

# Create a dictionary to store file hashes
file_hashes = defaultdict(list)

# Traverse the directory and calculate hashes for all files
for root, dirs, files in walk(directory):
    for file in files:
        file_path = path.join(root, file)
        file_hash = hash_file(file_path)
        file_hashes[file_hash].append(file_path)

# Find duplicate files
duplicates = {hash: paths for hash, paths in file_hashes.items() if len(paths) > 1}

# Remove duplicates
for paths in duplicates.values():
    # Sort by file length, and keep the shortest one
    paths.sort(key=len)
    for path in paths[1:]:
        # remove(path)
        print(f"Removed duplicate: {path}")

print("Duplicate removal complete.")


In [ ]:
find_and_remove_duplicate_files(dir_check_path)

In [ ]:
with ThreadPoolExecutor() as executor:
    executor.map(find_and_remove_duplicate_files, list_dir_check)